In [6]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
import sklearn.metrics as metrics
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
import os

In [9]:
# Process label 1 dataset
df_l1 = pd.read_csv('train_en_l1.csv')
df_l1 = df_l1.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs
df_l1[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = df_l1[
    ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_1' based on majority voting while ignoring NaNs
df_l1['label_1'] = (df_l1[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 1 (gendered abuse)
df_l1['binary_label_1'] = df_l1['label_1'].apply(lambda x: 'hate' if x == 1 else 'not_hate')

# Process label 3 dataset
df_l3 = pd.read_csv('train_en_l3.csv')
df_l3 = df_l3.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs
df_l3[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = df_l3[
    ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_3' based on majority voting while ignoring NaNs
df_l3['label_3'] = (df_l3[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 3 (explicit/aggressive)
df_l3['binary_label_3'] = df_l3['label_3'].apply(lambda x: 'explicit' if x == 1 else 'not_explicit')

# Select columns for merging
df_l1_slim = df_l1[['text', 'label_1', 'binary_label_1']]
df_l3_slim = df_l3[['text', 'label_3', 'binary_label_3']]

# Merge the datasets based on text field
merged_df = pd.merge(df_l1_slim, df_l3_slim, on='text', how='inner')

# Add a unique_id column to the merged dataset
merged_df['unique_id'] = [f'id_{i}' for i in range(len(merged_df))]

# Reorder columns
merged_df = merged_df[['unique_id', 'text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]

# Save the merged dataset
merged_df.to_csv('train_en_task3.csv', index=False)

# Display information
print(f"Final merged dataset shape: {merged_df.shape}")
print(merged_df.head())

# Check label distribution
print("\nLabel distribution:")
print(f"Label 1 (gendered abuse): {merged_df['binary_label_1'].value_counts().to_dict()}")
print(f"Label 3 (explicit/aggressive): {merged_df['binary_label_3'].value_counts().to_dict()}")

Final merged dataset shape: (6531, 6)
  unique_id                                               text binary_label_1  \
0      id_0      This is our history . For Shia and Sunni ,...       not_hate   
1      id_1     If this is "grilling" then wht wud u call t...           hate   
2      id_2    look at <handle replaced>madam. Locals attac...       not_hate   
3      id_3    our beloved yogi ji has given money home and...           hate   
4      id_4  "I gonna kill that bitch"  Bitvj imma kill you...           hate   

   label_1 binary_label_3  label_3  
0        0   not_explicit        0  
1        1       explicit        1  
2        0   not_explicit        0  
3        1       explicit        1  
4        1       explicit        1  

Label distribution:
Label 1 (gendered abuse): {'not_hate': 5159, 'hate': 1372}
Label 3 (explicit/aggressive): {'not_explicit': 4159, 'explicit': 2372}


In [10]:
import pandas as pd
import nltk
import string
import re

# Download necessary NLTK resources
nltk.download('stopwords')

# Set up tokenizer and stopwords
wpt = nltk.WordPunctTokenizer()
stop_words_init = nltk.corpus.stopwords.words('english')
stop_words = [i for i in stop_words_init if i not in ('not', 'and', 'for')]
print(stop_words)

# Function to normalize text for pre-processing
def normalize_text(text):
    text = str(text).lower()  # Ensure text is a string
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Load the multi-task dataset
merged_df = pd.read_csv('train_en_task3.csv')

# Apply text normalization
merged_df['processed_text'] = merged_df['text'].apply(lambda x: normalize_text(x))

# Further processing to remove '...'
merged_df['processed_text'] = merged_df['processed_text'].str.replace('...', '')

# Display samples of processed text
print("\nOriginal vs Processed Text Samples:")
for i in range(3):
    print(f"Original: {merged_df['text'].iloc[i]}")
    print(f"Processed: {merged_df['processed_text'].iloc[i]}")
    print("-" * 50)

# Keep all columns but add processed text
merged_df_final = merged_df[['unique_id', 'text', 'processed_text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]


print(f"\nProcessed dataset shape: {merged_df_final.shape}")
print(merged_df_final.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krmri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she'd", "she'll",

In [16]:
# Extract features (processed text)
X = list(merged_df_final['processed_text'])

# Extract labels for both tasks
y_task1 = merged_df_final['label_1'].values
y_task3 = merged_df_final['label_3'].values

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Activation, Dropout, Dense, Flatten,
    Bidirectional, GRU, concatenate, SpatialDropout1D,
    GlobalMaxPooling1D, GlobalAveragePooling1D, Conv1D,
    Embedding, Input, Concatenate
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import MeanSquaredError

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

######## Textual Features for Embedding ###################
max_len = 100
max_features = 4479

# Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

print(X)  # Check the processed sequences

[[  12    6   68 ...    0    0    0]
 [  34   12    6 ...    0    0    0]
 [ 158   64  239 ...    0    0    0]
 ...
 [  10   41  387 ...    0    0    0]
 [1590 1170   69 ...    0    0    0]
 [ 461  294   27 ...    0    0    0]]


In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_task1 = label_encoder.fit_transform(y_task1)
y_task3 = label_encoder.fit_transform(y_task3)

In [20]:
from keras.utils import to_categorical
y_task1 = to_categorical(y_task1, num_classes=2)
y_task3 = to_categorical(y_task3, num_classes=2)

In [21]:
import numpy as np
import json

# Load GloVe embeddings from JSON
with open('glove_embeddings.json', encoding="utf8") as f:
    embeddings_list = json.load(f)

# Convert the list of vectors to a dictionary with word indices as keys
embeddings_dictionary = {str(i): vector for i, vector in enumerate(embeddings_list)}

# Define tokenizer 
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
num_words = min(max_features, vocab_size)  # Limit vocab to max_features

# Get embedding dimension (from first vector in list)
embed_size = len(embeddings_list[0]) if embeddings_list else 0

# Initialize embedding matrix
embedding_matrix = np.zeros((num_words, embed_size))

# Fill embedding matrix with corresponding word vectors
for word, index in word_index.items():
    if index >= max_features:
        continue
    embedding_vector = embeddings_dictionary.get(word) or embeddings_dictionary.get(str(index))
    if embedding_vector is not None:
        embedding_matrix[index] = np.asarray(embedding_vector, dtype=np.float32)

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (4479, 50)


In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Embedding, SpatialDropout1D, Conv1D,
    Bidirectional, LSTM, Dense, Dropout,
    GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configure GPU for optimal performance
def configure_gpu():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth for each GPU
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
            # Use mixed precision for better performance
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            print('Mixed precision enabled')
        except RuntimeError as e:
            print(e)

configure_gpu()

# Multi-Task Model Definition
def create_multi_task_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size=300):
    """
    Creates a multi-task CNN-BiLSTM model architecture for joint prediction of
    gendered abuse (task1) and explicit language (task3)
    """
    # Input layer
    input_layer = Input(shape=(max_len,))
    
    # Embedding layer with pretrained weights (GloVe/FastText)
    embedding_layer = Embedding(
        input_dim=max_features,
        output_dim=embed_size,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False  # As per paper, embeddings are non-trainable
    )(input_layer)
    
    # Spatial Dropout to prevent overfitting
    spatial_dropout = SpatialDropout1D(0.2)(embedding_layer)
    
    # CNN Layer - shared representation
    conv_layer = Conv1D(
        filters=64,
        kernel_size=2,
        activation='relu',
        padding='same'
    )(spatial_dropout)
    
    # Bidirectional LSTM Layer - shared representation
    bilstm_layer = Bidirectional(
        LSTM(
            units=128,
            return_sequences=True,
            dropout=0.1,
            recurrent_dropout=0.1
        )
    )(conv_layer)
    
    # Global Average Pooling - shared representation
    gap_layer = GlobalAveragePooling1D()(bilstm_layer)
    
    # Task-specific layers for Task 1 (Gendered Abuse)
    task1_dense = Dense(128, activation='relu', name='task1_dense')(gap_layer)
    task1_dropout = Dropout(0.1)(task1_dense)
    task1_output = Dense(2, activation='softmax', name='task1_output', dtype='float32')(task1_dropout)
    
    # Task-specific layers for Task 3 (Explicit Language)
    task3_dense = Dense(128, activation='relu', name='task3_dense')(gap_layer)
    task3_dropout = Dropout(0.1)(task3_dense)
    task3_output = Dense(2, activation='softmax', name='task3_output', dtype='float32')(task3_dropout)
    
    # Create model with multiple outputs
    model = Model(inputs=input_layer, outputs=[task1_output, task3_output])
    
    return model

# Custom MacroF1Score Metric for multi-task learning
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes=2, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to predicted class indices
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convert one-hot encoded y_true to class indices if needed
        if len(y_true.shape) > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)
        
        # Initialize confusion matrix
        conf_matrix = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        
        # Calculate TP, FP, FN for each class
        diag = tf.linalg.diag_part(conf_matrix)
        row_sum = tf.reduce_sum(conf_matrix, axis=1)
        col_sum = tf.reduce_sum(conf_matrix, axis=0)
        
        tp = diag
        fp = col_sum - diag
        fn = row_sum - diag
        
        # Update the state variables
        self.tp.assign_add(tf.reduce_sum(tp))
        self.fp.assign_add(tf.reduce_sum(fp))
        self.fn.assign_add(tf.reduce_sum(fn))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        # Calculate precision and recall
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        
        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        
        # Return macro F1 (average of per-class F1 scores)
        return f1

    def reset_states(self):
        self.tp.assign(0.)
        self.fp.assign(0.)
        self.fn.assign(0.)
        self.count.assign(0.)

# Model Training for multi-task learning
def train_and_validate_multi_task_model(model, X_train, y_train_task1, y_train_task3, 
                                         X_val, y_val_task1, y_val_task3, 
                                         batch_size=32, epochs=15, model_dir='models_en_task3_m1'):
    """
    Trains the multi-task CNN-BiLSTM model with early stopping and model checkpointing
    Returns the best model and training history
    """
    # Create directory for saving models if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)
    
    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        mode='min',
        verbose=1
    )
    
    model_checkpoint = ModelCheckpoint(
        os.path.join(model_dir, 'best_model_en_task3_m1.h5'),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )
    
    # Compile model with Adam optimizer
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'task1_output': 'categorical_crossentropy',
            'task3_output': 'categorical_crossentropy'
        },
        loss_weights={
            'task1_output': 1.0,  # Weight for gendered abuse task
            'task3_output': 1.0   # Weight for explicit language task
        },
        metrics={
            'task1_output': ['accuracy', MacroF1Score(num_classes=2)],
            'task3_output': ['accuracy', MacroF1Score(num_classes=2)]
        }
    )
    
    # Train the model
    history = model.fit(
        X_train, 
        {'task1_output': y_train_task1, 'task3_output': y_train_task3},
        validation_data=(
            X_val, 
            {'task1_output': y_val_task1, 'task3_output': y_val_task3}
        ),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    # Load the best model found during training
    best_model = load_model(
        os.path.join(model_dir, 'best_model_en_task3_m1.h5'), 
        custom_objects={'MacroF1Score': MacroF1Score}
    )
    
    return history, best_model

# Plot Training History for multi-task model
def plot_multi_task_training_history(history, plot_dir='plots_en_task3_m1'):
    """
    Plots training history for both tasks (accuracy and loss curves)
    Saves plots to specified directory
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Task 1 (Gendered Abuse) plots
    plt.figure(figsize=(12, 10))
    
    # Plot task1 accuracy
    plt.subplot(2, 2, 1)
    plt.plot(history.history['task1_output_accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_task1_output_accuracy'], label='Validation Accuracy')
    plt.title('Task 1 (Gendered Abuse) Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot task1 loss
    plt.subplot(2, 2, 2)
    plt.plot(history.history['task1_output_loss'], label='Train Loss')
    plt.plot(history.history['val_task1_output_loss'], label='Validation Loss')
    plt.title('Task 1 (Gendered Abuse) Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Task 3 (Explicit Language) plots
    # Plot task3 accuracy
    plt.subplot(2, 2, 3)
    plt.plot(history.history['task3_output_accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_task3_output_accuracy'], label='Validation Accuracy')
    plt.title('Task 3 (Explicit Language) Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot task3 loss
    plt.subplot(2, 2, 4)
    plt.plot(history.history['task3_output_loss'], label='Train Loss')
    plt.plot(history.history['val_task3_output_loss'], label='Validation Loss')
    plt.title('Task 3 (Explicit Language) Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'training_history_en_task3_m1.png'))
    plt.close()
    
    # Plot combined loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Total Train Loss')
    plt.plot(history.history['val_loss'], label='Total Validation Loss')
    plt.title('Multi-Task Model Combined Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'combined_loss_en_task3_m1.png'))
    plt.close()

# Evaluation function for multi-task model
def evaluate_multi_task_validation(model, X_val, y_val_task1, y_val_task3, plot_dir='plots_en_task3_m1'):
    """
    Evaluates the multi-task model on validation data and saves metrics and plots
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Predict probabilities
    y_pred_task1, y_pred_task3 = model.predict(X_val, batch_size=32)
    
    # Convert to class labels
    y_pred_task1_labels = np.argmax(y_pred_task1, axis=1)
    y_true_task1_labels = np.argmax(y_val_task1, axis=1)
    
    y_pred_task3_labels = np.argmax(y_pred_task3, axis=1)
    y_true_task3_labels = np.argmax(y_val_task3, axis=1)
    
    # Task 1 (Gendered Abuse) metrics
    task1_precision = precision_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_recall = recall_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_weighted_f1 = f1_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_macro_f1 = f1_score(y_true_task1_labels, y_pred_task1_labels, average='macro')
    
    # Task 3 (Explicit Language) metrics
    task3_precision = precision_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_recall = recall_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_weighted_f1 = f1_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_macro_f1 = f1_score(y_true_task3_labels, y_pred_task3_labels, average='macro')
    
    # Classification reports
    task1_report = classification_report(y_true_task1_labels, y_pred_task1_labels, 
                                        target_names=['not_hate', 'hate'])
    
    task3_report = classification_report(y_true_task3_labels, y_pred_task3_labels, 
                                        target_names=['not_explicit', 'explicit'])
    
    # Confusion matrices
    task1_conf_matrix = confusion_matrix(y_true_task1_labels, y_pred_task1_labels)
    task3_conf_matrix = confusion_matrix(y_true_task3_labels, y_pred_task3_labels)
    
    # Plot confusion matrices
    # Task 1 confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(task1_conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Task 1 (Gendered Abuse) Confusion Matrix')
    plt.savefig(os.path.join(plot_dir, 'task1_confusion_matrix_en_task3_m1.png'))
    plt.close()
    
    # Task 3 confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(task3_conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Explicit', 'Explicit'],
                yticklabels=['Not Explicit', 'Explicit'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Task 3 (Explicit Language) Confusion Matrix')
    plt.savefig(os.path.join(plot_dir, 'task3_confusion_matrix_en_task3_m1.png'))
    plt.close()
    
    return {
        'task1': {
            'precision': task1_precision,
            'recall': task1_recall,
            'f1_score_weighted': task1_weighted_f1,
            'f1_score_macro': task1_macro_f1,
            'classification_report': task1_report,
            'confusion_matrix': task1_conf_matrix
        },
        'task3': {
            'precision': task3_precision,
            'recall': task3_recall,
            'f1_score_weighted': task3_weighted_f1,
            'f1_score_macro': task3_macro_f1,
            'classification_report': task3_report,
            'confusion_matrix': task3_conf_matrix
        }
    }

# Main Execution for Training and Validation
if __name__ == "__main__":
    # Split into train (80%) and validation (20%)
    X_train, X_val, y_train_task1, y_val_task1, y_train_task3, y_val_task3 = train_test_split(
        X, y_task1, y_task3, test_size=0.2, random_state=42, stratify=y_task1
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    
    # Create multi-task model
    embed_size = embedding_matrix.shape[1]
    model = create_multi_task_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size)
    
    # Print model summary
    model.summary()
    
    # Train multi-task model
    history, trained_model = train_and_validate_multi_task_model(
        model, X_train, y_train_task1, y_train_task3, 
        X_val, y_val_task1, y_val_task3,
        batch_size=32,
        epochs=15  
    )
    
    # Plot training history
    plot_multi_task_training_history(history)
    
    # Evaluate on validation set
    val_results = evaluate_multi_task_validation(
        trained_model, X_val, y_val_task1, y_val_task3
    )
    
    # Print Task 1 (Gendered Abuse) results
    print("\nTask 1 (Gendered Abuse) Validation Results:")
    print(f"Precision: {val_results['task1']['precision']:.4f}")
    print(f"Recall: {val_results['task1']['recall']:.4f}")
    print(f"weighted F1 Score: {val_results['task1']['f1_score_weighted']:.4f}")
    print(f"macro F1 Score: {val_results['task1']['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['task1']['classification_report'])
    
    # Print Task 3 (Explicit Language) results
    print("\nTask 3 (Explicit Language) Validation Results:")
    print(f"Precision: {val_results['task3']['precision']:.4f}")
    print(f"Recall: {val_results['task3']['recall']:.4f}")
    print(f"weighted F1 Score: {val_results['task3']['f1_score_weighted']:.4f}")
    print(f"macro F1 Score: {val_results['task3']['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['task3']['classification_report'])

Training samples: 5224
Validation samples: 1307


C:\Users\krmri\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 100, 50)           │         223,950 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d             │ (None, 100, 50)           │               0 │ embedding[0][0]            │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 100, 64)           │           6,464 │ spatial_dropout1d[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 100, 256)          │         197,632 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 256)               │               0 │ bidirectional[0][0]        │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task1_dense (Dense)           │ (None, 128)               │          32,896 │ global_average_pooling1d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task3_dense (Dense)           │ (None, 128)               │          32,896 │ global_average_pooling1d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ task1_dense[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ task3_dense[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task1_output (Dense)          │ (None, 2)                 │             258 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task3_output (Dense)          │ (None, 2)                 │             258 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 494,354 (1.89 MB)

...

164/164 ━━━━━━━━━━━━━━━━━━━━ 26s 110ms/step - loss: 1.2264 - task1_output_accuracy: 0.7829 - task1_output_loss: 0.5550 - task1_output_macro_f1_score: 0.7829 - task3_output_accuracy: 0.6226 - task3_output_loss: 0.6714 - task3_output_macro_f1_score: 0.6226 - val_loss: 1.2061 - val_task1_output_accuracy: 0.7896 - val_task1_output_loss: 0.5336 - val_task1_output_macro_f1_score: 0.7896 - val_task3_output_accuracy: 0.6389 - val_task3_output_loss: 0.6749 - val_task3_output_macro_f1_score: 0.6389
Epoch 2/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 1.1543 - task1_output_accuracy: 0.7966 - task1_output_loss: 0.5059 - task1_output_macro_f1_score: 0.7966 - task3_output_accuracy: 0.6451 - task3_output_loss: 0.6484 - task3_output_macro_f1_score: 0.6451 
Epoch 2: val_loss improved from 1.20606 to 1.16738, saving model to models_en_task3_m1\best_model_en_task3_m1.h5


164/164 ━━━━━━━━━━━━━━━━━━━━ 17s 106ms/step - loss: 1.1544 - task1_output_accuracy: 0.7965 - task1_output_loss: 0.5059 - task1_output_macro_f1_score: 0.7965 - task3_output_accuracy: 0.6451 - task3_output_loss: 0.6485 - task3_output_macro_f1_score: 0.6451 - val_loss: 1.1674 - val_task1_output_accuracy: 0.7888 - val_task1_output_loss: 0.5163 - val_task1_output_macro_f1_score: 0.7888 - val_task3_output_accuracy: 0.6396 - val_task3_output_loss: 0.6525 - val_task3_output_macro_f1_score: 0.6396
Epoch 3/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 1.1592 - task1_output_accuracy: 0.7894 - task1_output_loss: 0.5146 - task1_output_macro_f1_score: 0.7894 - task3_output_accuracy: 0.6487 - task3_output_loss: 0.6446 - task3_output_macro_f1_score: 0.6487  
Epoch 3: val_loss improved from 1.16738 to 1.15461, saving model to models_en_task3_m1\best_model_en_task3_m1.h5


164/164 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step - loss: 1.1592 - task1_output_accuracy: 0.7894 - task1_output_loss: 0.5145 - task1_output_macro_f1_score: 0.7894 - task3_output_accuracy: 0.6487 - task3_output_loss: 0.6446 - task3_output_macro_f1_score: 0.6487 - val_loss: 1.1546 - val_task1_output_accuracy: 0.7995 - val_task1_output_loss: 0.5071 - val_task1_output_macro_f1_score: 0.7995 - val_task3_output_accuracy: 0.6404 - val_task3_output_loss: 0.6486 - val_task3_output_macro_f1_score: 0.6404
Epoch 4/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 1.1612 - task1_output_accuracy: 0.7943 - task1_output_loss: 0.5099 - task1_output_macro_f1_score: 0.7943 - task3_output_accuracy: 0.6429 - task3_output_loss: 0.6513 - task3_output_macro_f1_score: 0.6429  
Epoch 4: val_loss improved from 1.15461 to 1.15061, saving model to models_en_task3_m1\best_model_en_task3_m1.h5


164/164 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step - loss: 1.1612 - task1_output_accuracy: 0.7943 - task1_output_loss: 0.5099 - task1_output_macro_f1_score: 0.7943 - task3_output_accuracy: 0.6429 - task3_output_loss: 0.6512 - task3_output_macro_f1_score: 0.6429 - val_loss: 1.1506 - val_task1_output_accuracy: 0.7995 - val_task1_output_loss: 0.5065 - val_task1_output_macro_f1_score: 0.7995 - val_task3_output_accuracy: 0.6488 - val_task3_output_loss: 0.6452 - val_task3_output_macro_f1_score: 0.6488
Epoch 5/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 1.1588 - task1_output_accuracy: 0.7933 - task1_output_loss: 0.5092 - task1_output_macro_f1_score: 0.7933 - task3_output_accuracy: 0.6350 - task3_output_loss: 0.6497 - task3_output_macro_f1_score: 0.6350  
Epoch 5: val_loss did not improve from 1.15061
164/164 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step - loss: 1.1587 - task1_output_accuracy: 0.7933 - task1_output_loss: 0.5091 - task1_output_macro_f1_score: 0.7933 - task3_output_accuracy: 0.6351 - ta

41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step  

Task 1 (Gendered Abuse) Validation Results:
Precision: 0.8080
Recall: 0.7995
weighted F1 Score: 0.7233
macro F1 Score: 0.4980

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.80      1.00      0.89      1032
        hate       0.84      0.06      0.11       275

    accuracy                           0.80      1307
   macro avg       0.82      0.53      0.50      1307
weighted avg       0.81      0.80      0.72      1307


Task 3 (Explicit Language) Validation Results:
Precision: 0.7168
Recall: 0.6488
weighted F1 Score: 0.5243
macro F1 Score: 0.4245

Classification Report:
              precision    recall  f1-score   support

not_explicit       0.65      1.00      0.78       835
    explicit       0.84      0.03      0.07       472

    accuracy                           0.65      1307
   macro avg       0.74      0.52      0.42      1307
weighted avg       0.72      0.65      0.52      1307


In [27]:
# Process label 1 dataset
df_l1 = pd.read_csv('test_en_l1.csv')
df_l1 = df_l1.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs
df_l1[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = df_l1[
    ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_1' based on majority voting while ignoring NaNs
df_l1['label_1'] = (df_l1[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 1 (gendered abuse)
df_l1['binary_label_1'] = df_l1['label_1'].apply(lambda x: 'hate' if x == 1 else 'not_hate')

# Process label 3 dataset
df_l3 = pd.read_csv('test_en_l3.csv')
df_l3 = df_l3.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs
df_l3[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = df_l3[
    ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_3' based on majority voting while ignoring NaNs
df_l3['label_3'] = (df_l3[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 3 (explicit/aggressive)
df_l3['binary_label_3'] = df_l3['label_3'].apply(lambda x: 'explicit' if x == 1 else 'not_explicit')

# Select columns for merging
df_l1_slim = df_l1[['text', 'label_1', 'binary_label_1']]
df_l3_slim = df_l3[['text', 'label_3', 'binary_label_3']]

# Merge the datasets based on text field
merged_df = pd.merge(df_l1_slim, df_l3_slim, on='text', how='inner')

# Add a unique_id column to the merged dataset
merged_df['unique_id'] = [f'id_{i}' for i in range(len(merged_df))]

# Reorder columns
merged_df = merged_df[['unique_id', 'text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]

# Save the merged dataset
merged_df.to_csv('test_en_task3.csv', index=False)

# Display information
print(f"Final merged dataset shape: {merged_df.shape}")
print(merged_df.head())

# Check label distribution
print("\nLabel distribution:")
print(f"Label 1 (gendered abuse): {merged_df['binary_label_1'].value_counts().to_dict()}")
print(f"Label 3 (explicit/aggressive): {merged_df['binary_label_3'].value_counts().to_dict()}")

Final merged dataset shape: (1107, 6)
  unique_id                                               text binary_label_1  \
0      id_0  "who's this tweet about" bitch it's about you ...       not_hate   
1      id_1  #AlkaLamba should’ve used her slippers to slap...       not_hate   
2      id_2  #CoronaVillains  #COVIDIDIOT   #COVID2019 is a...           hate   
3      id_3  (Acts 29:19994) LIKE the female "governors" of...           hate   
4      id_4  ***Let us celebrate “secular” as a beautiful w...       not_hate   

   label_1 binary_label_3  label_3  
0        0   not_explicit        0  
1        0   not_explicit        0  
2        1       explicit        1  
3        1       explicit        1  
4        0   not_explicit        0  

Label distribution:
Label 1 (gendered abuse): {'not_hate': 877, 'hate': 230}
Label 3 (explicit/aggressive): {'not_explicit': 699, 'explicit': 408}


In [32]:
merged_df = pd.read_csv('test_en_task3.csv')

# Apply text normalization
merged_df['processed_text'] = merged_df['text'].apply(lambda x: normalize_text(x))

# Further processing to remove '...'
merged_df['processed_text'] = merged_df['processed_text'].str.replace('...', '')

# Display samples of processed text
print("\nOriginal vs Processed Text Samples:")
for i in range(3):
    print(f"Original: {merged_df['text'].iloc[i]}")
    print(f"Processed: {merged_df['processed_text'].iloc[i]}")
    print("-" * 50)

# Keep all columns but add processed text
merged_df_final = merged_df[['unique_id', 'text', 'processed_text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]


print(f"\nProcessed dataset shape: {merged_df_final.shape}")
print(merged_df_final.head())

# Extract features (processed text)
X = list(merged_df_final['processed_text'])

# Extract labels for both tasks
y_task1 = merged_df_final['label_1'].values
y_task3 = merged_df_final['label_3'].values
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

y_task1 = label_encoder.fit_transform(y_task1)
y_task3 = label_encoder.fit_transform(y_task3)

y_task1 = to_categorical(y_task1, num_classes=2)
y_task3 = to_categorical(y_task3, num_classes=2)


Original vs Processed Text Samples:
Original: "who's this tweet about" bitch it's about you fucking kiss me already
Processed: who s this tweet about bitch it s about you fucking kiss me already
--------------------------------------------------
Original: #AlkaLamba should’ve used her slippers to slap that misogynistic idiot.
Processed: alkalamba should’ve used her slippers to slap that misogynistic idiot
--------------------------------------------------
Original: #CoronaVillains  #COVIDIDIOT   #COVID2019 is a #PandemicCovid19   Don't spread your #FakeNewsAlert promoting #FakeCases  #womenaretrash
Processed: coronavillains covididiot is a don t spread your fakenewsalert promoting fakecases womenaretrash
--------------------------------------------------

Processed dataset shape: (1107, 7)
  unique_id                                               text  \
0      id_0  "who's this tweet about" bitch it's about you ...   
1      id_1  #AlkaLamba should’ve used her slippers to slap...   


In [33]:
# Evaluate on validation set
test_results = evaluate_multi_task_validation(
    trained_model, X, y_task1, y_task3
)

# Print Task 1 (Gendered Abuse) results
print("\nTask 1 (Gendered Abuse) Test Results:")
print(f"Precision: {test_results['task1']['precision']:.4f}")
print(f"Recall: {test_results['task1']['recall']:.4f}")
print(f"weighted F1 Score: {test_results['task1']['f1_score_weighted']:.4f}")
print(f"macro F1 Score: {test_results['task1']['f1_score_macro']:.4f}")
print("\nClassification Report:")
print(test_results['task1']['classification_report'])

# Print Task 3 (Explicit Language) results
print("\nTask 3 (Explicit Language) Test Results:")
print(f"Precision: {test_results['task3']['precision']:.4f}")
print(f"Recall: {test_results['task3']['recall']:.4f}")
print(f"weighted F1 Score: {test_results['task3']['f1_score_weighted']:.4f}")
print(f"macro F1 Score: {test_results['task3']['f1_score_macro']:.4f}")
print("\nClassification Report:")
print(test_results['task3']['classification_report'])

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step

Task 1 (Gendered Abuse) Test Results:
Precision: 0.6264
Recall: 0.7850
weighted F1 Score: 0.6968
macro F1 Score: 0.4398

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.79      0.99      0.88       877
        hate       0.00      0.00      0.00       230

    accuracy                           0.79      1107
   macro avg       0.40      0.50      0.44      1107
weighted avg       0.63      0.79      0.70      1107


Task 3 (Explicit Language) Test Results:
Precision: 0.5836
Recall: 0.6314
weighted F1 Score: 0.4952
macro F1 Score: 0.3962

Classification Report:
              precision    recall  f1-score   support

not_explicit       0.63      0.99      0.77       699
    explicit       0.50      0.01      0.02       408

    accuracy                           0.63      1107
   macro avg       0.57      0.50      0.40      1107
weighted avg       0.58      0.63      0.50      1107

